

***
<div class="alert alert-block alert-info">

# Multimedia fugacity models
# – LEVEL I –
Authors: Antonia Praetorius (a.praetorius@uva.nl) and Marianne Seijo (m.x.seijo@uva.nl) <br><br>
</div>

Copyright: 
_The present codes are a suport for teaching and stay under the copyright of the authors_<br>
Use of the software constitutes your agreement to abide by the terms and conditions set out in the license agreement.
Models are provided "as is" with no guarantees, as described in the License agreements. Users are urged to read the License agreement provided with each model prior to installing or using the software. And, although these models are provided as "freeware", in fairness to the developers, the funding agencies and partners, and to all users, they are copyright.    
    

Databases within the models are not intended as an authoritative information source and all physical chemical properties should be verified by the user. Users should be especially careful about the temperature dependence of physical chemical properties.    
    


***

<div class="alert alert-block alert-success">
    
## Learning Objectives:
The goal of this suite of models is to have a first interaction with environmental fate modelling using multimedia models and understand the connections between physico-chemical properties of chemicals and their environmental fate.
    
</div>    

***

<div class="alert alert-block alert-success">
    

## Lesson Reminders: 
    

### Level I 
    Simple equilibrium partitioning calculations
    
A Level I simulation is designed to model the following:
    
- A closed environmental system
- A constant amount of chemical present in the system
- System is in equilibrium
- Only considers thermodynamic drivers of fate (equilibrium partitioning) of a fixed quantity of conserved (ie. non-reacting) chemical
    

This Level 1 model is set up as a so-called "unit world" mass-balance model. These models are a very simplified representation of the environment, but can be used as evaluative tools, e.g. for comparative assessment of different chemicals or for gaining insights into the relative importance of different fate processes. In a unit world model the environment is represented as a set of well-mixed chemical reactors (boxes), each representing one environmental medium (here: air, water, soil and sediments).

In a Level 1 model (simplest model) we only consider the equilibrium distribution of a fixed quantity of conserved chemical (no reactions/degradation, no transport out of the system). It is irrelevant to which medium the chemical is emitted, as it is assumed to become instantaneously distributed between the different media to reach equilibrium. A Level 1 model ignores a lot of relevant fate processes (to be added in Levels 2 and 3), but provides a general impression of which media the chemical will likely accumulate in and what relative concentrations can be expected in different compartments.
    
The model is based on the fugacity concept (only applicable to organic chemicals). Fugacity, f, describes “fleeing tendency” of a compound (urge of a molecules to escape or flee from one phase to another. Fugacity is the partial pressure which a chemical exerts as it attempts to escape from one phase and migrate to another (units: Pa). The fugacity approach is convenient for environmental fate models, as fugacity-based models are mathematically simpler than concentration-based models, easier to interpret (equilibrium status between the phases immediately obvious because at equilibrium fugacities are equal) and it conveys how close system is to equilibrium and what the direction of diffuse transfer processes is.
See slides for more details on fugacity, Z values and their relationship to partition coefficients. 

</div>    
    
    
<br><br>



***
## Level I Modelisation step by step: 




We will work with the following example: 


![LEVEL1_Figure](img/LEVEL1_Figure.png)
<br>





# Python library used here: 
## Prerequisites:

Tested with version: 
Python version 3.9 </br>
Pandas version 1.5.1</br>
Ipywidgets:  8.0.2 </br>


In [ ]:
import sys

restart = False
    
try:
    # pandas is used to read excel file
    import pandas as pd
    print("pandas="+str(pd.__version__))
except ImportError:
    print ('pandas is not installed, installing it now.The kernel will restart. ')
    !{sys.executable} -m pip install pandas
    restart = True
 
    
try:
    # plotly is used to create map and data on it
    import plotly
    print("plotly="+str(plotly.__version__))
except ImportError:
    print ('plotly is not installed, installing it now.The kernel will restart. ')
    !{sys.executable} -m pip install plotly
    restart = True  
    
import math   


try:
    # ipywidgets is used to create map and data on it
    import ipywidgets
    print("ipywidgets="+str(ipywidgets.__version__))
except ImportError:
    print ('ipywidgets is not installed, installing it now.The kernel will restart. ')
    !{sys.executable} -m pip install ipywidgets
    restart = True  


***
***
# Input Parameters

## Constants

In [ ]:
R = 8.314  # gas constant (Pa.m3/mol K)

## Environmental inputs:

In [ ]:
# Surface area (m2): ------------------------
S_Air      = 1.0e11
S_Soil     = 10.0e10
S_Water    = 9.0e10
S_Sediment = 9.0e10

# Depth (m): --------------------------
depth_Air      = 1.0e3
depth_Soil     = 0.2
depth_Water    = 20.0
depth_Sediment = 0.05

# Volume (m3): ------------------------
V_Air      = S_Air * depth_Air
V_Soil     = S_Soil * depth_Soil
V_Water    = S_Water * depth_Water
V_Sediment = S_Sediment * depth_Sediment


# Density (kg.m-3): --------------------
density_Air      = 1.185413
density_Soil     = 2400.0
density_Water    = 1000.0
density_Sediment = 2400.0

density_Aerosol        = 2000.0
density_Susp_Particles = 1500.0
density_Fish           = 1000.0

# Volume Fractions: ---------------------------
Vf_Aerosol_In_Air          = 2.0e-11
Vf_Susp_Particles_In_Water = 5.0e-3
Vf_Fish_In_Water           = 1.0e-3   
Vf_Aerosol_In_Air         = 2.0e-6

# Lipid In Fish: (L.kg-1)---------------------------
L_Lipid_In_Fish           = 0.05  

# Fraction of Organic Carbon for solid (g.g-1) ---------------
foc_Suspended_Particles = 0.15
foc_Soil                = 0.1
foc_BottomSediment      = 0.05


## Chemicals Properties Inputs

In [ ]:
Name                = "1,4-Dichlorobenzene"
Molar_Mass          = 147.00      # M  (g.mol-1)
Data_Temperature    = 25.0        # T  (°C)
Melting_Point       = 53.5        # Tm (°C)
Vapor_Pressure      = 170.0       # P (Pa) 
Solubility_In_Water = 79.0        # S (g.m-3) 
logKOW              = 3.4         

# Henry's law constant:  
# take fromcChemicals Properties Inputs if available, otherwise estimate as stated here
# Comment the unneeded line (by adding # in the beginning of the unneeded line), uncomment the one needed

# Henry_Constant = 244 #(Pa·m3.mol-1)
Henry_Constant = Molar_Mass * Vapor_Pressure / Solubility_In_Water #(Pa·m3.mol-1)


# Level I : Total mass of chemical in environment : ---------
massChem            = 20.0         # m (kg) 


***
***

# Calculations
***
## Partition Coefficients Calculations



| Compartments                |     Partition Coefficient (L.kg-1)                    
|:-----------------------------|:------------------------------------------------------------
| **(1) Air-Water**              | ${K_{AW} = \frac{H}{(R*T)}}$                          
| **(2) Water-Sediment**         | K$_{SedW}$ = K$_{OC}$ *foc$_{BottomSediment}$ 
| **(3) Water-Soil**             | K$_{SoilW}$ = K$_{OC}$ *foc$_{Soil}$ 
| **(4) Air-Soil**              | ${K_{SoilA}=\frac{K_{SoilW}*\rho_{Soil}}{K_{AW}*\rho_{Water}}}$  
| **(5) Fish-Water**             | ${K_{FW} = K_{OW} * L}$
| **(6) Susp.Particles-Water**    | K$_{Susp.Part.W}$ = K$_{OC}$ *foc$_{Susp. Part.}$
| **(7) Aerosol-Air**            | ${K_{AeA} = \frac{6.10^{6}} { P_L}}$





With:    
    
**H**, the Henry's law constant ($Pa.m^3.mol^{-1}$)     
**R**, the ideal gas constant = 8.314 $Pa.m^3.mol^{-1}.K$    
**T**, the temperature in Kelvin (K)

**K$_{OC}$**, the organic carbon-water partition coefficient     
**f$_{OC}$**, the fraction of organic carbon in solid (g.g-1)    

**$\rho_{Soil}$** = density of the solid phase S (kg.L$^{-1}$)    
**$\rho_{Water}$** = density of water (or biota) (kg.L$^{-1}$)    

K$_{OW}$, the octanol-water partition coefficient   
L : fish lipids content

$P_L$: Subcooled Liquid Vapor Pressure.   
&emsp;For Tm the melting point temperature, T the temperature and P the Gas Pressure, 
if T>Tm: ${P_L = P.exp^{[6.79.((Tm/T)-1)]}}$ 


#### Organic Carbon-Water Partition Coefficient : 
The K$_{OC}$ depends on the nature of the medium (soil composition etc.) and the chemical component: <br>
cf https://setac.onlinelibrary.wiley.com/doi/pdf/10.1897/01-362 <br>
Two equations are proposed here: <br>
- **=> <ins> Karickhoff method</ins> : $\boxed{K_{OC_K} = 0.41 * K_{OW}}$** <br>
Karickhoff SW. 1984. Organic pollutant sorption in aquatic sys-tems.J Hydraul Eng Am Soc Civil Eng110:707–735  <br>
cf: https://doi.org/10.1061/(ASCE)0733-9429(1984)110:6(707) <br><br>

- **=> <ins> Seth et al. method</ins> : $\boxed{K_{OC_S} = 0.35 * K_{OW}}$** <br>
Seth  R,  Mackay  D,  Muncke  J.  1999.  Estimating  the  organiccarbon  partition  coefficient and  its  variability for hydrophobicchemicals.Environ Sci Technol33:2390–2394.  <br>
cf: https://pubs.acs.org/doi/full/10.1021/es980893j <br>


In [ ]:


# Air-water partition coefficient (KAW) ------------------------------------------ 
KAW = Henry_Constant / (R *  (Data_Temperature + 273.15)) 

# Octanol-water partition coefficient 
#******* Calculate from logKOW ***********- 
KOW = pow(10.0,logKOW)         
                         
# Organic Carbon-Water Partition Coefficient (KOC): ------------------------------ 
# The KOC depend on the nature of soil and the chemical components: cf https://setac.onlinelibrary.wiley.com/doi/pdf/10.1897/01-362
# Two equations are proposed here. Other may be added depending on the chemical componant.
KOC_K = 0.41 * KOW # (L.kg-1) Karickhoff method. Karickhoff SW. 1984. Organic pollutant sorption in aquatic sys-tems.J Hydraul Eng Am Soc Civil Eng110:707–735                                                       
KOC_S = 0.35 * KOW # (L.kg-1) Seth et al. method hydrophobic organics. Seth  R,  Mackay  D,  Muncke  J.  1999.  Estimating  the  organiccarbon  partition  coefficient and  its  variability for hydrophobicchemicals.Environ Sci Technol33:2390–2394.                                        

#Selected KOC for the chemical:
KOC = KOC_K

# Fish-water partition coefficient (KFW): --------------------------------
KFW = L_Lipid_In_Fish * KOW                         
                         
# Suspended Particle-Water Partition Coefficient (KSPW): --------------------------------
KSPW = KOC * foc_Suspended_Particles

# Soil-Water Partition Coefficient (KSW): --------------------------------
KSW = KOC * foc_Soil

# Sediment-Water Partition Coefficient (KSedWat): --------------------------------
KSedW = KOC * foc_BottomSediment 


# Soil-Air Partition Coefficient (KSA): --------------------------------
KSA = KSW * density_Soil / (KAW * density_Water )


# Aerosol-Air Partition Coefficient (KAeA): --------------------------------
KAeA = 0.0

if (Data_Temperature<Melting_Point):
    tempRatio = (Melting_Point+273.15)/(Data_Temperature+273.15)-1
    PL = Vapor_Pressure * math.exp(6.79*tempRatio)
    KAeA = 6e6/PL


#Saved results in a dataframe    
dataPCoef = {'Partition Coefficient':  ['K_Air-Water', 
                                   'K_Soil-Air', 
                                   'K_Soil-Water',
                                   'K_Sediment-Water',
                                   'K_Fish-Water',
                                   'K_Suspended Particle-Water',
                                   'K_Aerosol-Air',     
                                  ],
               'Value': [KAW , KSA, KSW, KSedW, KFW, KSPW, KAeA]
        }

dfPC = pd.DataFrame(dataPCoef)
print('Partition coefficients calculated:')
display(dfPC)


***
## Z and VZ

Different way to compute Z with partition coefficients previously calculated.     
We choose the following:     

| Compartments                 |     Definition of Z (mol.m-3.Pa)                 |      
|:-----------------------------|:----------------------|
| **Air**                      | ${K_{AW}/H}$ =1/(R*T)              |          
| **Water**                    | ${1/H}$        |
| **Soil**                     | ${K_{SW}/H}$          |
| **Sediments**                | ${K_{SedW}/H}$        |
| **Fish**                     | ${K_{FishW}/H}$       |
| **Susp.Particles**           | ${K_{SPW}/H}$         |
| **Aerosol**                  | ${K_{AeA}*Z_{Air}}$   |    


In [ ]:
ZAir = KAW/Henry_Constant
ZWater = 1.0/Henry_Constant
ZSoil = KSW/Henry_Constant
ZSediment = KSedW/Henry_Constant
ZFish = KFW/Henry_Constant
ZSuspendedParticles = KSPW/Henry_Constant
ZAerosol = KAeA*ZAir

VZAir = ZAir * V_Air * (1.0 - Vf_Aerosol_In_Air)
VZWater = ZWater * V_Water*(1.0 - Vf_Fish_In_Water - Vf_Susp_Particles_In_Water)
VZSoil = ZSoil * V_Soil
VZSediment = ZSediment * V_Sediment
VZFish = ZFish * V_Water * Vf_Fish_In_Water
VZSuspendedParticles = ZSuspendedParticles * V_Water*Vf_Susp_Particles_In_Water
VZAerosol = ZAerosol* V_Air * Vf_Aerosol_In_Air

TotalVZ = VZAir + VZWater + VZSoil + VZSediment + VZFish + VZSuspendedParticles + VZAerosol
print("Total sum (VZ) = ", TotalVZ, "")


***
## Fugacity in Level I


### Fugacity (in Pa) = Total mol of chemical in environment / $\sum{(VZ)}$


In [ ]:
Fugacity = (1000.0  * massChem / Molar_Mass) / TotalVZ
print("Fugacity of",Name, " = ", Fugacity, "Pa")


***
## Concentrations in each medium:

### C$_{mediaX}$ = Molar_Mass * Fugacity * Z$_{mediaX}$ 

In [ ]:
# Concentrations (g.m-3) ----------------
CAir = Molar_Mass * Fugacity * ZAir
CWater = Molar_Mass * Fugacity * ZWater
CSoil = Molar_Mass * Fugacity * ZSoil
CSediment = Molar_Mass * Fugacity * ZSediment
CFish = Molar_Mass * Fugacity * ZFish
CSuspendedParticles = Molar_Mass * Fugacity * ZSuspendedParticles
CAerosol = Molar_Mass * Fugacity * ZAerosol


# Amounts (kg) ----------------
mm = Molar_Mass / 1000.0
AAir = mm * Fugacity * VZAir
AWater = mm * Fugacity * VZWater
ASoil = mm * Fugacity * VZSoil 
ASediment = mm * Fugacity * VZSediment
AFish = mm * Fugacity * VZFish 
ASuspendedParticles = mm * Fugacity * VZSuspendedParticles
AAerosol = mm * Fugacity * VZAerosol

# Percentage ----------------
pAir = 100.0 * AAir / massChem
pWater = 100.0 * AWater / massChem
pSoil = 100.0 * ASoil / massChem
pSediment = 100.0 * ASediment / massChem
pFish = 100.0 * AFish / massChem
pSuspendedParticles = 100.0 * ASuspendedParticles / massChem
pAerosol = 100.0 * AAerosol / massChem



#Save results in a dataframe:     

dataZ = {'Name':   ['Level I', 
                'Level I', 
                'Level I',
                'Level I',
                'Level I',
                'Level I',
                'Level I'                      
               ],
    'Compartments':   ['Air', 
                'Air',
                'Water', 
                'Water',
                'Water',                      
                'Soil',
                'Sediment'            
               ],
        'Specificity':   ['Dissolved in Air', 
                          'Aerosol',
                          'Dissolved in Water', 
                          'Fish',
                          'Suspended Particle',
                          'Soil',
                          'Sediment'                         
               ],         
         
               'ZValue (mol.m-3·Pa)': [ZAir, 
                                      ZAerosol,
                                      ZFish,
                                      ZSuspendedParticles,
                                      ZWater,
                                      ZSoil,
                                      ZSediment
                ],
                'VZ (mol/Pa)': [VZAir, 
                                VZAerosol,
                                VZWater,
                                VZFish,
                                VZSuspendedParticles,
                                VZSoil,
                                VZSediment],  
         
                'Concentrations (g.m-3)': [CAir, 
                                CAerosol,          
                                CWater,
                                CFish,
                                CSuspendedParticles,
                                CSoil,
                                CSediment],
         
                'Amounts (kg)':[AAir, 
                                AAerosol,
                                AWater,
                                AFish,
                                ASuspendedParticles,
                                ASoil,
                                ASediment],
                '%': [pAir, 
                      pAerosol,
                      pWater,
                      pFish,
                      pSuspendedParticles,
                      pSoil,
                      pSediment]        
        }
dfZ = pd.DataFrame(dataZ)
display(dfZ)

# Test on mass conservation
print("Input massChem = ", massChem)
sumAmount = dfZ['Amounts (kg)'].sum()
print("Total sum of amounts = ", sumAmount)
if (massChem == sumAmount):
    print("The mass is conserved")
else:
    print("The mass is not conserved. A problem occured.")



***
***
## Plot data: 

In [ ]:
from ipywidgets import widgets


grid = widgets.GridspecLayout(20, 6)



bTitle = widgets.Button(description="Level I: "+Name+" at equilibrium", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = 'white', font_weight='bold', text_color='black',font_size="20"))
grid[0:1, 0:] = bTitle


bTitle2 = widgets.Button(description="Fugacity: {0:e}".format(Fugacity)+" Pa; ", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = 'white', font_weight='bold', text_color='black',font_size="20"))
grid[1:2, 0:] = bTitle2

bAir = widgets.Button(description="Air: Total Mass = {0:6.3f}".format(AAir+AAerosol)+" kg"+
                                   " ;   {0:3.2f}".format(pAir+pAerosol)+" % of the Total", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = 'lightblue', font_weight='bold', text_color='black',font_size="20")
                     )
grid[2:5, 0:] = bAir

bSoil = widgets.Button(description="Soil: Total Mass = {0:6.3f}".format(ASoil)+" kg"+
                                   " ;   {0:3.2f}".format(pSoil)+" % of the Total", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = '#c5672c', font_weight='bold', text_color='white',font_size="24")
                      )
grid[5:-1, 0:2] = bSoil

bWater = widgets.Button(description="Water: Total Mass = {0:6.3f}".format(AWater+ASuspendedParticles+AFish)+" kg"+
                                   " ;   {0:3.2f}".format(pWater+pSuspendedParticles+pFish)+" % of the Total", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = '#4994ec', font_weight='bold', text_color='white',font_size="24")
)
grid[5:16, 2:] = bWater 


bSediment = widgets.Button(description="Sediments: Total Mass = {0:6.3f}".format(ASediment)+" kg"+
                                   " ;   {0:3.2f}".format(pSediment)+" % of the Total", 
                                   layout=widgets.Layout(height="auto", width="auto"),
                                   style=dict(button_color = '#dbad3a', font_weight='bold', text_color='white',font_size="24")
)

grid[16:-1, 2:] = bSediment


display(grid)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

values = '%'

fig1 = px.sunburst(dfZ, title = "Level I Distribution of " + Name + ' ' + values, 
                  path=['Name','Compartments','Specificity'], values=values)
fig1.show()


In [ ]:
fig2 = px.bar(dfZ, title="Level I Distribution of "+Name+' '+values, 
              x='Compartments', y=values, color='Specificity', text_auto=True)
fig2.show()

***
## Save results in a html file: 

In [ ]:
import json

from ipywidgets import IntSlider
from ipywidgets.embed import embed_data


data = embed_data(views=[grid])

html_template = """
<!-- Load RequireJS, used by the IPywidgets for dependency management -->
<script 
    src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" 
    integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" 
    crossorigin="anonymous">
</script>

<!-- Load IPywidgets bundle for embedding. -->
<script
      data-jupyter-widgets-cdn="https://unpkg.com/"
      data-jupyter-widgets-cdn-only
      src="https://cdn.jsdelivr.net/npm/@jupyter-widgets/html-manager@*/dist/embed-amd.js" 
      crossorigin="anonymous">
</script>

<!-- The state of all the widget models on the page -->
<script type="application/vnd.jupyter.widget-state+json">
      {manager_state}
</script>

<div id="first-slider-widget">
<!-- This script tag will be replaced by the view's DOM tree -->
    <script type="application/vnd.jupyter.widget-view+json">
        {widget_views[0]}
    </script>
</div>

"""
manager_state = json.dumps(data['manager_state'])
widget_views = [json.dumps(view) for view in data['view_specs']]
rendered_template = html_template.format(manager_state=manager_state, widget_views=widget_views)



with open('Level1_'+Name+'_'+'.html', 'w') as fp:
    fp.write(dfZ.to_html())
    fp.write(fig1.to_html(full_html=False, include_plotlyjs='cdn'))
    fp.write(fig2.to_html(full_html=False, include_plotlyjs='cdn'))
    fp.write(rendered_template)
fp.close()